In [1]:
import pickle

import glob2
import datetime

import numpy as np
from utils.data_reading.sound_data.station import StationsCatalog

/home/plerolland/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [6]:
datasets_yaml = "/home/plerolland/Bureau/dataset.yaml"
year = 2021
stations_c = StationsCatalog(datasets_yaml).filter_out_undated().filter_out_unlocated()

# output
detections_file = f"../../data/detections/{year}/detections.npy"

In [7]:
to_read = glob2.glob(f"../../data/detections/{year}/*.p")

detections = {}
for file in to_read:
    info = file.split("_")[-1][:-2].split("-")
    name, year_station = "-".join(info[:-1]), int(info[-1])
    station = stations_c.by_starting_year(year_station).by_names(name)[0]
    detections[station] = []
    with open(file, "rb") as f:
        while True:
            try:
                detections[station].append(pickle.load(f))
            except EOFError:
                break
                
    detections[station] = np.array(detections[station])
    detections[station] = detections[station][np.argsort(detections[station][:,0])]

    # merge close detections
    to_del = set()
    i, j = 0, 0
    while j<len(detections[station]):
        delta = (np.abs(detections[station][i,0] - detections[station][j,0]))
        if delta.total_seconds() < 5:
            detections[station][i,0] += delta / 2
            to_del.add(j)
        else:
            i = j
        j+=1
    detections[station] = np.delete(detections[station], list(to_del), axis=0)
    
    print(f"{len(list(to_del))} lines deleted ; {len(detections[station])} left for station {station}")
np.save(detections_file, detections)

23 lines deleted ; 500854 left for station station_ELAN_2020
11 lines deleted ; 178775 left for station station_MADE_2021
6 lines deleted ; 244905 left for station station_MADW_2020
16 lines deleted ; 399564 left for station station_NEAMS_2020
16 lines deleted ; 463027 left for station station_RTJ_2020
49 lines deleted ; 343767 left for station station_SSEIR_2020
7 lines deleted ; 195930 left for station station_SWAMS_2020
19 lines deleted ; 381016 left for station station_WKER2_2020
2 lines deleted ; 54311 left for station station_H01W1_2021
2 lines deleted ; 78180 left for station station_H04N1_2021
1 lines deleted ; 30801 left for station station_H04S1_2021
3 lines deleted ; 50710 left for station station_H08S1_2021
